# Machine Learning 5 - Lineer Sınıflandırıcılar, Lojistik Regresyon, SVM

Merhabalar, Machine Learning serimizin beşincisinde;
* Doğrusal sınıflandırıcıları tanıyacak,
* Lojistik regresyon ve SVM kavramlarını örneklendirerek inceleyeceğiz.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    RocCurveDisplay,
)
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

## Doğrusal Sınıflandırma Nedir?

Yazı dizimizin üçüncüsünde, **Doğrusal Regresyon** konusuna değinmiştik. Sonrasında da bu metodu, bir ev fiyatı tahminleyicisi oluşturmak için kullanmıştık. **Doğrusal(Lineer) Regresyon** ile **Mantıksal(Lojistik) Regresyon** kavramları birbirine karıştırılmamalıdır.

**Lojistik Regresyon**, bir **Doğrusal Sınıflandırıcı**'dır. Bir sınıfı belirleyen bir veya daha fazla bağımsız değişken bulunan bir veri kümesini analiz etmek için kullanılan istatistiksel bir yöntemdir. Sonuç, ikili bir değişkenle ölçülür (yalnızca iki olası sonuç vardır). O örnek, aradığımız sınıfa aittir veya değildir(Diğer sınıflardan birine aittir).

## Karar Sınırı, Doğrusal Sınıflandırıcı, Doğrusal Ayrılabilirlik

Teorik anlatıma devam etmeden önce, bir **Doğrusal Sınıflandırıcı** görsel olarak neye benzer, nasıl çalışır bunu görelim.

In [2]:
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html

from sklearn.datasets import make_blobs
# generate dataset
X_example, y_example = make_blobs(n_samples=1000, centers=2, n_features=2, random_state=1, cluster_std=3)
# define bounds of the domain
min1, max1 = X_example[:, 0].min()-1, X_example[:, 0].max()+1
min2, max2 = X_example[:, 1].min()-1, X_example[:, 1].max()+1
# define the x and y scale
x1grid = np.arange(min1, max1, 0.1)
x2grid = np.arange(min2, max2, 0.1)
# create all of the lines and rows of the grid
xx, yy = np.meshgrid(x1grid, x2grid)
# flatten each grid to a vector
r1, r2 = xx.flatten(), yy.flatten()
r1, r2 = r1.reshape((len(r1), 1)), r2.reshape((len(r2), 1))
# horizontal stack vectors to create x1,x2 input for the model
grid = np.hstack((r1,r2))
# define the model
model = LogisticRegression()
# fit the model
model.fit(X_example, y_example)
# make predictions for the grid
yhat = model.predict(grid)
# reshape the predictions back into a grid
zz = yhat.reshape(xx.shape)
# plot the grid of x, y and z values as a surface
plt.contourf(xx, yy, zz, cmap='Paired')
# create scatter plot for samples from each class
for class_value in range(2):
    # get row indexes for samples with this class
    row_ix = np.where(y_example == class_value)
    # create scatter of these samples
    plt.scatter(X_example[row_ix, 0], X_example[row_ix, 1], cmap='Paired')

NameError: name 'np' is not defined